<a href="https://colab.research.google.com/github/rattlesczck/Chatbots/blob/main/AI_agent_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#using langgraph to build the AI agent

# Why? Langgraph?
# --> Scalable
# --> Flexible
# --> fault tolerance
# --> Granular level logic definition

# archituecture
# ---> 1) user query --> 2)chatbot interacts with LLM and gets output --> 3) gives the output
# Langgraph creates a flow in the form of a graph itself, same as the above mentioned architcure




In [2]:
!pip install langgraph
!pip install langsmith
!pip install langchain langchain_groq langchain_community



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.7 MB/s eta 0:00:00


In [4]:
from google.colab import userdata

groq_api_key = userdata.get('chatbot_groq')
# print(groq_api_key)
langsmith_api_key = userdata.get('chatbot_langsmith')
# print(langsmith_api_key)

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


In [6]:
import os
os.environ["LANGCHAIN_API_KEY"] = langsmith_api_key
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "ChatbotAgent"

In [7]:
from langchain_groq import ChatGroq


In [8]:
llm = ChatGroq(groq_api_key = groq_api_key, model_name = "Gemma2-9b-It")
llm



ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7b32ad85df50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7b32ad8bded0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [9]:
# Start building the chatbot using Langgraph
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages #as messages are added, the state of the graph keeps changing



In [22]:
class State(TypedDict):
  # there is a variable called "messages", which is of the type "Annotated". it says tht it is a list type
  # and we use a function "add_messages" which is for appending messages to the list

  messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

In [11]:
graph_builder

In [23]:
def chatbot(state:State):
  #After we get any response from the chatbot, we have to update the "messages" variable
  return {"messages":llm.invoke(state['messages'])}



In [24]:
graph_builder.add_node("chatbot", chatbot)

In [25]:
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

In [26]:
graph = graph_builder.compile()

In [38]:
while (True):
  user_input = input("User:")
  if user_input.lower() in ["quit", "exit"]:
    print("Bye!")
    break
  for event in graph.stream({"messages":("user", user_input)}):
    #print(event.values())
    for value in event.values():
      #print(value['messages'])
      print("Assistant:", value['messages'].content)

User:hello
Assistant: Hello! 👋

How can I help you today? 😊

User:what is the purpose of humanity?
Assistant: As a large language model, I don't have personal beliefs or the ability to determine the purpose of humanity. That's a complex philosophical question that humans have been pondering for centuries.

There isn't a single, universally accepted answer. Different cultures, religions, and individuals have their own perspectives on what gives life meaning. Here are some common viewpoints:

* **To find happiness and fulfillment:** Many believe the purpose of life is to experience joy, love, and a sense of purpose.
* **To make a difference in the world:** Some find meaning in contributing to society, helping others, or leaving a positive impact.
* **To learn and grow:** Others see life as a journey of self-discovery and personal development.
* **To connect with something greater:** Some find purpose in their faith or in a sense of connection to nature or the universe.

Ultimately, the p